In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [44]:
#fake text classification
df = pd.read_csv('/content/drive/My Drive/MY ML JOURNEY/Deep Learning Practice/Dataset/fake text classification/train.csv')

In [45]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [46]:
df=df.dropna()
X=df.drop('label',axis=1)
y=df['label']

In [47]:
X.shape,y.shape

((18285, 4), (18285,))

In [48]:
import tensorflow as tf

In [49]:
tf.__version__

'2.3.0'

In [50]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional,Dense

In [51]:
voc_size=5000
txt = X.copy()
txt['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [52]:
txt.reset_index(inplace=True)

In [53]:
txt.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [54]:
import nltk
import re
from nltk.corpus import stopwords

In [55]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(txt)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', txt['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
onehot=[one_hot(words,voc_size)for words in corpus] 
onehot

In [59]:
sent_length=20
embedded=pad_sequences(onehot,padding='pre',maxlen=sent_length)
print(embedded)

[[   0    0    0 ... 1999 1493 1114]
 [   0    0    0 ... 4314 4979 3021]
 [   0    0    0 ... 2154 4869 1662]
 ...
 [   0    0    0 ... 3553  391 1963]
 [   0    0    0 ... 4535 3788 3314]
 [   0    0    0 ... 2004  528 4022]]


In [60]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [61]:
len(embedded),y.shape

(18285, (18285,))

In [62]:
X_final=np.array(embedded)
y_final=np.array(y)

In [63]:
X_final[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3062,
        686, 2904, 2684, 1333, 3618, 3007, 1999, 1493, 1114], dtype=int32)

In [64]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [66]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 2s 11ms/step - loss: 0.3257 - accuracy: 0.8481 - val_loss: 0.1966 - val_accuracy: 0.9165
Epoch 2/10
192/192 [==============================] - 2s 9ms/step - loss: 0.1415 - accuracy: 0.9426 - val_loss: 0.1960 - val_accuracy: 0.9175
Epoch 3/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0980 - accuracy: 0.9631 - val_loss: 0.2413 - val_accuracy: 0.9127
Epoch 4/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0759 - accuracy: 0.9715 - val_loss: 0.2693 - val_accuracy: 0.9140
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0594 - accuracy: 0.9798 - val_loss: 0.3046 - val_accuracy: 0.9084
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0427 - accuracy: 0.9863 - val_loss: 0.4160 - val_accuracy: 0.9102
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0346 - accuracy: 0.9891 - val_loss: 0.4270 - val_accuracy: 0.9046
Epoch

In [67]:

from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [68]:
y_pred=model.predict_classes(X_test)

In [69]:
from sklearn.metrics import confusion_matrix

In [70]:
confusion_matrix(y_test,y_pred)

array([[1243, 2176],
       [2107,  509]])

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.2903065451532726